<a href="https://colab.research.google.com/github/Fuenfgeld/DMA2023TeamD/blob/main/Code/ReHealthCo_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ReHealthCo ETL-Prozess

## Vorbereitung 

###CleanUp - Eventuell gespeicherte Variablen löschen vor Ausführung des Colab books (Ziel Reproduzierbarkeit)

In [ ]:
# Löschen aller Variablen
%reset -f

## Importieren der notwendigen Libraries: sqlite3, csv, pandas

In [ ]:
import sqlite3
from sqlite3 import Error
import csv
import pandas as pd
from pandas.util import hash_pandas_object

### Erstellung einer virtuellen Kopie des GitHub Repositorium DMA2023TeamD in Colab

In [ ]:
!git clone https://github.com/Fuenfgeld/DMA2023TeamD.git

In [ ]:
!pwd

In [ ]:
# Wechsel in den Github-Projekt-Ordner
%cd /content/DMA2023TeamD/Code

In [ ]:
# Auflistung aller Ordnerinhalte
!ls

### Verknüpfung G-Dive mit Colab

In [ ]:
!mkdir -p /content/drive
from google.colab import drive
drive.mount('/content/drive')

### Benennung der rhc-DataFrames und Einlesen der entsprechenden csv-Files

In [ ]:
# Erstellung von Pfad-Abkürzungen
material_path = "/content/drive/Shareddrives/Projektarbeit DMA Gruppe D/"
csv_path_rhc = f"{material_path}csv_data/rhc"

**✈ !!! In nächster Code Zelle ggf eigenen Materialpfad angeben !!!**

In [ ]:
# Einlesen der csv-Files in die Dataframes
rhcpat_df = pd.read_csv(f"{csv_path_rhc}/rhcpat_df.csv")
rhccon_df = pd.read_csv(f"{csv_path_rhc}/rhccon_df.csv")
rhcmed_df = pd.read_csv(f"{csv_path_rhc}/rhcmed_df.csv")
rhcenc_df = pd.read_csv(f"{csv_path_rhc}/rhcenc_df.csv")

In [ ]:
print(rhcpat_df)

# Einfügen einer neuen Spalte mit tod/lebendig

In [ ]:
# Spalte DEATHDATE verdoppeln
rhcpat_df.loc[:, 'LEBENDIG'] = rhcpat_df.loc[:, 'DEATHDATE']
# Anzeigen alle aktuellen Spalten
print(rhcpat_df.columns.tolist())

## Ändern des Inhalts der Spalte GENDER

### (da dieser wegen String-Inhalten Konflikte auslöst in der zukünftigen Prediction )

In [ ]:
# Anzeige des Spalteninhalts GENDER
rhcpat_df["GENDER"]

## Python replace() method to update values in a dataframe

In [ ]:
rhcpat_df['GENDER'].replace("F", 1, inplace = True)
rhcpat_df['GENDER'].replace("M", 0, inplace = True)
rhcpat_df['GENDER']

# Todesdatum nullwerte = lebendig

### convert "LEBENDIG" to datetime

In [ ]:
# NaN Werte in Spalte LEBENDIG in 1 (= lebend) umschreiben und aller vorhandenen Daten in 0 (= tot)
rhcpat_df["LEBENDIG"] = pd.to_datetime(rhcpat_df["LEBENDIG"])
today = pd.Timestamp.today().floor('d')
rhcpat_df.loc[rhcpat_df['LEBENDIG'] < today, 'LEBENDIG'] = "0"
rhcpat_df['LEBENDIG'] = rhcpat_df['LEBENDIG'].fillna("1")
rhcpat_df.head(12)

# calculate age in Tabelle rhcpat_df

# fill nall values with todays date

In [ ]:
rhcpat_df["DEATHDATE"] = rhcpat_df.DEATHDATE.fillna(pd.to_datetime("today"))

# convert to datetime

In [ ]:
rhcpat_df["DEATHDATE"] = pd.to_datetime(rhcpat_df["DEATHDATE"])
rhcpat_df["BIRTHDATE"] = pd.to_datetime(rhcpat_df["BIRTHDATE"])
rhcpat_df["AGE"] = rhcpat_df.DEATHDATE.dt.year - rhcpat_df.BIRTHDATE.dt.year
# Anzeige des Ergebnis
rhcpat_df.head()

### Erstellung einer INDEX-SPALTE für **RHC_PSEUDONYM** für Tabelle Patients

In [ ]:
rhcpatidx_df = rhcpat_df.rename_axis('RHC_PSEUDONYM').reset_index() # Benennung der neuen Spalte mit RHC_PSEUDONYM
print(rhcpatidx_df) # Anzeige des Ergebnis

### Erstellung einer **Schlüsseltabelle** und Erstellung einer INDEX-SPALTE für RHC_PSEUDONYM

In [ ]:
rhcpat_schluesseltabelle_df = rhcpat_df[["Id"]] # Neue Tabelle Schlüssetabelle beinhaltet nur die Spalte Id und den Index
rhcpat_schluesseltabelleidx_df = rhcpat_df.rename_axis('RHC_PSEUDONYM').reset_index() # Index benennen als RHC_PSEUDONYM
print(rhcpat_schluesseltabelleidx_df) # Anzeige des Ergebnis

# Erstellung Dauer stationäre Krankenhaustage

In [ ]:
rhcenc_df["START"] = pd.to_datetime(rhcenc_df["START"])
rhcenc_df["STOP"] = pd.to_datetime(rhcenc_df["STOP"])
rhcenc_df["BEHANDLUNGSDAUER"] = rhcenc_df.STOP - rhcenc_df.START
rhcenc_df['BEHANDLUNGSDAUER'] = rhcenc_df['BEHANDLUNGSDAUER'].dt.total_seconds()/(3600.0)
rhcenc_df.sort_values(by='BEHANDLUNGSDAUER', ascending=False) # Zeitangabe in Stunden

# Erstellung Dauer der Conditions

In [ ]:
rhccon_df["START"] = pd.to_datetime(rhccon_df["START"])
rhccon_df["STOP"] = pd.to_datetime(rhccon_df["STOP"])
rhccon_df["CONDITIONDAUER"] = rhccon_df.STOP - rhccon_df.START
rhccon_df['CONDITIONDAUER'] = rhccon_df['CONDITIONDAUER'].dt.total_seconds()/(3600.0)
rhccon_df.sort_values(by='CONDITIONDAUER', ascending=False) # Zeitangabe in Stunden

# Tabellenvereinigung

In [ ]:
rhc_verein1_df = pd.DataFrame.merge(rhcpatidx_df, rhccon_df, left_on=["Id", "DATENBANKART"], right_on=["PATIENT", "DATENBANKART"], how="left")
rhc_verein1_df.drop("PATIENT", axis=1, inplace=True)
rhc_verein1_df.rename(columns={"DESCRIPTION": "desc_con", "CODE": "code_con", "START": "start_con", "STOP": "stop_con"}, inplace=True)
# Überprüfung ob RHC_PSEUDONYM numerisch aufsteigt (oder ob Patienten rausgeflogen sind)
rhc_verein1_df.sort_values(by=['RHC_PSEUDONYM']).head(12)

In [ ]:
rhc_verein2_df = pd.DataFrame.merge(rhc_verein1_df, rhcmed_df, left_on=["Id", "DATENBANKART", "ENCOUNTER"], right_on=["PATIENT", "DATENBANKART", "ENCOUNTER"], how="left")
rhc_verein2_df.drop("PATIENT", axis=1, inplace=True)
rhc_verein2_df.rename(columns={"DESCRIPTION": "desc_med", "CODE": "code_med", "START": "start_med", "STOP": "stop_med", "REASONDESCRIPTION": "reasondesc_med", "REASONCODE": "reasoncode_med"}, inplace=True)
rhc_verein2_df.head(12)

In [ ]:
rhcenc_df.rename(columns={"Id": "id_enc"}, inplace=True)
print(rhcenc_df)

In [ ]:
rhc_vereinall_df = pd.DataFrame.merge(rhc_verein2_df, rhcenc_df, left_on=["Id", "DATENBANKART", "PAYER", "PAYER_COVERAGE"], right_on=["PATIENT", "DATENBANKART", "PAYER", "PAYER_COVERAGE"], how="left")
rhc_vereinall_df.drop("PATIENT", axis=1, inplace=True)
rhc_vereinall_df.rename(columns={"DESCRIPTION": "desc_enc", "CODE": "code_enc", "START": "start_enc", "STOP": "stop_enc", "REASONDESCRIPTION": "reasondesc_enc", "REASONCODE": "reasoncode_enc"}, inplace=True)
rhc_vereinall_df.head(12) # Überprüfung des Inhalts

In [ ]:
# Anzeige aller Zeilen
rhc_vereinall_df.shape[0]

In [ ]:
# Anzeige aller Spalten
rhc_vereinall_df.shape[1]

# Tabelle um Spalten reduzieren zur Anonymisierung und
## dann Spalten mit präfix pat/con/med/enc benennen zur Zuordnung der Ursprungstabelle

In [ ]:
# Anzeigen alle aktuellen Spalten
print(rhc_vereinall_df.columns.tolist())

In [ ]:
# drop
rhc_vereinall_df.drop(["Id", "BIRTHDATE", "DEATHDATE", "SSN", "DRIVERS", "PASSPORT", "PREFIX", "FIRST", "LAST", "SUFFIX", "MAIDEN", "MARITAL", "RACE", "ETHNICITY", "BIRTHPLACE", "ADDRESS", "CITY", "STATE", "COUNTY", "ZIP", "LAT", "LON", 'ENCOUNTER', 'start_med', 'stop_med', 'PAYER', 'code_med', 'desc_med', 'BASE_COST', 'PAYER_COVERAGE', 'DISPENSES',  'reasoncode_med', 'reasondesc_med', 'id_enc', 'start_enc', 'stop_enc', 'ORGANIZATION', 'PROVIDER', 'code_enc', 'desc_enc', 'BASE_ENCOUNTER_COST', 'TOTAL_CLAIM_COST', 'reasoncode_enc', 'reasondesc_enc', 'start_con', 'stop_con'], axis=1, inplace=True)
print(rhc_vereinall_df.columns.tolist())

In [ ]:
rhc_vereinall_df.rename(columns={"GENDER": "GENDER_PAT", "HEALTHCARE_EXPENSES": "HEALTHCARE_EXPENSES_PAT", "HEALTHCARE_COVERAGE": "HEALTHCARE_COVERAGE_PAT", "datenbankart": "DATENBANKART", "code_con": "CODE_CON", "desc_con": "DESC_CON", "TOTALCOST": "TOTALCOST_MED", "ENCOUNTERCLASS": "ENCOUNTERCLASS_ENC"}, inplace=True)
print(rhc_vereinall_df.columns.tolist())

In [ ]:
rhc_vereinall_df.groupby(["DESC_CON"]).size()

Bereinigung der Spalte *DESC_CON* auf Diagnosen

In [ ]:
# Löschen bestimmter Spaltenwerte
anonym_rhcpat_df = rhc_vereinall_df.drop(rhc_vereinall_df[ (rhc_vereinall_df['DESC_CON'] == "Wheezing (finding)") |
                                                     (rhc_vereinall_df['DESC_CON'] == "Vomiting symptom (finding)") |
                                                     (rhc_vereinall_df['DESC_CON'] == "Suspected COVID-19") |
                                                     (rhc_vereinall_df['DESC_CON'] == "Sputum finding (finding)") |
                                                     (rhc_vereinall_df['DESC_CON'] == "Sore throat symptom (finding)") |
                                                     (rhc_vereinall_df['DESC_CON'] == "Respiratory distress (finding)") |
                                                     (rhc_vereinall_df['DESC_CON'] == "Passive conjunctival congestion (finding)") |
                                                     (rhc_vereinall_df['DESC_CON'] == "Nausea (finding)") |
                                                     (rhc_vereinall_df['DESC_CON'] == "Nasal congestion (finding)") |
                                                     (rhc_vereinall_df['DESC_CON'] == "Muscle pain (finding)") |
                                                     (rhc_vereinall_df['DESC_CON'] == "Loss of taste (finding)") |
                                                     (rhc_vereinall_df['DESC_CON'] == "Joint pain (finding)") |
                                                     (rhc_vereinall_df['DESC_CON'] == "History of myocardial infarction (situation)") |
                                                     (rhc_vereinall_df['DESC_CON'] == "History of cardiac arrest (situation)") |
                                                     (rhc_vereinall_df['DESC_CON'] == "Hemoptysis (finding)") |
                                                     (rhc_vereinall_df['DESC_CON'] == "Headache (finding)") |
                                                     (rhc_vereinall_df['DESC_CON'] == "Fever (finding)") |
                                                     (rhc_vereinall_df['DESC_CON'] == "Fatigue (finding)") |
                                                     (rhc_vereinall_df['DESC_CON'] == "Dyspnea (finding)") |
                                                     (rhc_vereinall_df['DESC_CON'] == "Diarrhea symptom (finding)") |
                                                     (rhc_vereinall_df['DESC_CON'] == "Cough (finding)") |
                                                     (rhc_vereinall_df['DESC_CON'] == "Chill (finding)")
                                                     ].index)
anonym_rhcpat_df.groupby(["DESC_CON"]).size()

In [ ]:
anonym_rhcpat_df

In [ ]:
# Anzeige aller Zeilen
anonym_rhcpat_df.shape[0]

In [ ]:
# Anzeige aller Spalten
anonym_rhcpat_df.shape[1]

# dataframe in sql schreiben

### Erstellung der Funktion create_connection weleche eine Verbindung mit der SQL-Datenbank herstellt

In [ ]:
# Connect to source database
def create_connection(db_file):
  """ create a database connection to SQLite database"""
  conn = None
  try:
    conn = sqlite3.connect(db_file)
    return conn
  except Error as error:
    print("Error while connecting to sqlite", error)
  # finally:
  #   if conn:
  #     conn.close()

Erstellung der (leeren) anonymisierten Datenbanken RHC

In [ ]:
# create a temporary empty anonym_rhc_database in the memory
anonym_rhc_db = f"{material_path}/source_dbs/anonym_rehealthco.db"
anonym_rhc_conn = create_connection(anonym_rhc_db)
anonym_rhc_cur = anonym_rhc_conn.cursor()
anonym_rhc_conn.commit()
print("Database connection is successfully")

In [ ]:
anonym_rhcpat_df.to_sql(
    "RHC",# Name of SQL table.
    anonym_rhc_conn, # sqlalchemy.engine.Engine or sqlite3.Connection
    schema=None, # Something can't understand yet. just keep it.
    if_exists='replace', # How to behave if the table already exists. You can use 'replace', 'append' to replace it.
    index=True, # It means index of DataFrame will save. Set False to ignore the index of DataFrame.
    index_label="Lft_Nr", # Depend on index. 
    chunksize=None, # Just means chunksize. If DataFrame is big will need this parameter.
    dtype=None, # Set the columns type of sql table.
    method=None, # Unstable. Ignore it.
)

In [ ]:
# Cleanup
anonym_rhc_conn.commit()

In [ ]:
# Check data in RHC

anonym_rhc_cur.execute("SELECT * FROM RHC;")
rows = anonym_rhc_cur.fetchall()
for row in rows:
 print(row)

Export csv file ReHealthCo Rohdatenbank in g-drive

In [ ]:
anonym_rhcpat_df.to_csv(f"{csv_path_rhc}/anonym_rhcpat.csv", index=False)
rhcpat_schluesseltabelleidx_df[["RHC_PSEUDONYM", "Id"]].to_csv(f"{csv_path_rhc}/rhc_keytbl.csv", index=False)

In [ ]:
%cd f"{csv_path_rhc}"

In [ ]:
!ls

# Version Check

In [ ]:
#python Version
import sys
sys.version_info

In [ ]:
pd.__version__

In [ ]:
csv.__version__

In [ ]:
# Überprüfen aller Variablen
# %pip freeze

# Calculate the Checksum of the Data Set

In [ ]:
rhcpat_df_hashes = hash_pandas_object(rhcpat_df)
rhccon_df_hashes = hash_pandas_object(rhccon_df)
rhcmed_df_hashes = hash_pandas_object(rhcmed_df)
rhcenc_df_hashes = hash_pandas_object(rhcenc_df)
rhcpatidx_df_hashes = hash_pandas_object(rhcpatidx_df)
rhcpat_schluesseltabelle_df_hashes = hash_pandas_object(rhcpat_schluesseltabelle_df)
rhcpat_schluesseltabelleidx_df_hashes = hash_pandas_object(rhcpat_schluesseltabelleidx_df)
rhc_verein1_df_hashes = hash_pandas_object(rhc_verein1_df)
rhc_verein2_df_hashes = hash_pandas_object(rhc_verein2_df)
rhc_vereinall_df_hashes = hash_pandas_object(rhc_vereinall_df)
anonym_rhcpat_df_hashes = hash_pandas_object(anonym_rhcpat_df)

### Erstellung einer Liste mit allen DF-hashes

In [ ]:
Liste_rhcetcall_hashes = [rhcpat_df_hashes, rhccon_df_hashes, rhcmed_df_hashes, rhcenc_df_hashes, rhcpatidx_df_hashes, rhcpat_schluesseltabelle_df_hashes, rhcpat_schluesseltabelleidx_df_hashes, rhc_verein1_df_hashes, rhc_verein2_df_hashes, rhc_vereinall_df_hashes, anonym_rhcpat_df_hashes]

In [ ]:
print(Liste_rhcetcall_hashes)

## Value should be 
-2151775413921651613

-1042772926083362217

6124240394299011028

-4704486740659085250

2822202601526760987

-3225392982667861325

2822202601526760987

4153508358133212794

-38465912529792371

-1472503524631281063

930312348553314926

In [ ]:
print(rhcpat_df_hashes.sum())
print(rhccon_df_hashes.sum())
print(rhcmed_df_hashes.sum())
print(rhcenc_df_hashes.sum())
print(rhcpatidx_df_hashes.sum())
print(rhcpat_schluesseltabelle_df_hashes.sum())
print(rhcpat_schluesseltabelleidx_df_hashes.sum())
print(rhc_verein1_df_hashes.sum())
print(rhc_verein2_df_hashes.sum())
print(rhc_vereinall_df_hashes.sum())
print(anonym_rhcpat_df_hashes.sum())